# Programming for Data Analytics in the Higher Diploma in Science in Data Analytics.

## Project_Superstore

**by Grainne Boyle**

This notebook contains a project that demonstrates what I have learned in this module.

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
#To ignore warnings, re: the figure layout changes, we import the warnings module.  
import warnings
warnings.filterwarnings('ignore')


In [2]:
# I had difficulty reading in the file, I was getting an error. I used chardet, a library that can detect file encodings.

#The file looked okay but there may have been unreadable characters, meaning the file was encoded with one format and the application is trying to read it using a different encoding format.
import chardet  # This imports a library that can detect file encodings

# Opens your file in binary mode ('rb') to read the raw bytes
with open('sample_superstore.csv', 'rb') as file:
    raw_data = file.read()  # Reads the entire file as raw bytes
    result = chardet.detect(raw_data)  # Analyzes the bytes to guess the encoding
    encoding = result['encoding']  # Gets the detected encoding type
    
# Uses the detected encoding to read the CSV file correctly. I used Chatgpt to find out how to encode the file and this solution was suitable.
storedf = pd.read_csv('sample_superstore.csv', encoding=encoding)

In [3]:
print(storedf.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

In [4]:
# Change the columns to datetime
storedf['Order Date'] = pd.to_datetime(storedf['Order Date'], format='%m/%d/%Y')
storedf['Ship Date'] = pd.to_datetime(storedf['Ship Date'], format='%m/%d/%Y') 

In [5]:
# From viewing the data in excel, I decided to remove the following columns as there was either too much detail to analyse, e.g Customer ID or not enough varied detail , e.g Country is only United States.  
storedf = storedf.drop(columns=['Row ID', 'Order ID', 'Customer ID', 'Customer Name', 'Country', 'Postal Code', 'Product ID'])

In [6]:
# after viewing in excel, I thought about what columns I could add to broaden my analysis:

# This added a column that calculates the time taken to process the order, how many days after the order was taken before it shipped. 
storedf.insert(loc=2, column='Order Processing Days', value=(storedf['Ship Date'] - storedf['Order Date']).dt.days)

# This enters a column that shows the month only so I can see if there are higher or lower sales in certain months.

storedf.insert(loc=3, column='Month', value=storedf['Order Date'].dt.month)

# This enters a column that shows the year only. 
storedf.insert(loc=4, column='Year', value=storedf['Order Date'].dt.year)
 

# This enters a column that shows the gross  as a percentage of sales.

storedf['Profit Margin (%)'] = (storedf['Profit'] / storedf['Sales']) * 100

# Note - I am assuming the sales figure is after the discount has been given. 
# This enters a column showing the sales price per unit(assuming after the discount)

storedf.insert(loc=15, column='SP per unit', value= (storedf['Sales'] / storedf['Quantity']))

# I am creating a function to create the final column. This column will classify the sales price per unit into a low,medium or high value category.  
def sales_cat(value):
    if value < 100:
        return "Low value"
    elif value > 1000:
        return "High value"
    else:
        return "Medium value"
    
storedf['Sales Category'] = storedf['SP per unit'].apply(sales_cat)





In [19]:

# Setting the format of the numerical columns:def set_display_format():
def set_display_format():
    pd.set_option('display.float_format', '{:,.2f}'.format)

# Call the function at the start of your script
set_display_format()

In [20]:
# Next, I preview the data to identify any issues that may arise and pre-process it if necessary.


# I run the list again to see what is included now and ensure that it included the columns in the correct locations:  
print(storedf.head())

# This shows the amount of rows and columns:  
storedf.shape



  Order Date  Ship Date  Order Processing Days  Month  Year       Ship Mode  \
0 2016-11-08 2016-11-11                      3     11  2016    Second Class   
1 2016-11-08 2016-11-11                      3     11  2016    Second Class   
2 2016-06-12 2016-06-16                      4      6  2016    Second Class   
3 2015-10-11 2015-10-18                      7     10  2015  Standard Class   
4 2015-10-11 2015-10-18                      7     10  2015  Standard Class   

     Segment             City       State Region         Category  \
0   Consumer        Henderson    Kentucky  South        Furniture   
1   Consumer        Henderson    Kentucky  South        Furniture   
2  Corporate      Los Angeles  California   West  Office Supplies   
3   Consumer  Fort Lauderdale     Florida  South        Furniture   
4   Consumer  Fort Lauderdale     Florida  South  Office Supplies   

  Sub-Category                                       Product Name  Sales  \
0    Bookcases                  Bu

(9994, 20)

In [21]:
# This shows the data types, I have converted the date columns to datetime, the other columns contain objects, integers and floats.
print(storedf.dtypes)


Order Date               datetime64[ns]
Ship Date                datetime64[ns]
Order Processing Days             int64
Month                             int32
Year                              int32
Ship Mode                        object
Segment                          object
City                             object
State                            object
Region                           object
Category                         object
Sub-Category                     object
Product Name                     object
Sales                           float64
Quantity                          int64
SP per unit                     float64
Discount                        float64
Profit                          float64
Profit Margin (%)               float64
Sales Category                   object
dtype: object


In [22]:
# This describes the numerical dataset:

storedf.describe(include=[np.number])

# This is useful as it highlights the main statistics. It would seem that there is no missing data, they all have 9994 rows , I will do another check on this below.   
# The order processing days average is 3.95 or 4.  
# The average selling price is $60.91 but the std deviation is $142.92 and the minimum is $0.33 showing a significant variability.
# Ths highest sales value is $22,638 and the highest selling price per unit is $3,773 which could suggest large outliers compared to the mean.
# The average discount is 16%, the minimum is 0% so not all sales are discounted, and the highest discount is 80%.  

,Order Processing Days,Month,Year,Sales,Quantity,SP per unit,Discount,Profit,Profit Margin (%)
count,"9,994.00","9,994.00","9,994.00","9,994.00","9,994.00","9,994.00","9,994.00","9,994.00","9,994.00"
mean,3.96,7.81,"2,015.72",229.86,3.79,60.92,0.16,28.66,12.03
std,1.75,3.28,1.12,623.25,2.23,142.93,0.21,234.26,46.68
min,0.00,1.00,"2,014.00",0.44,1.00,0.34,0.00,"-6,599.98",-275.00
25%,3.00,5.00,"2,015.00",17.28,2.00,5.47,0.00,1.73,7.50
50%,4.00,9.00,"2,016.00",54.49,3.00,16.27,0.20,8.67,27.00
75%,5.00,11.00,"2,017.00",209.94,5.00,63.94,0.20,29.36,36.25
max,7.00,12.00,"2,017.00","22,638.48",14.00,"3,773.08",0.80,"8,399.98",50.00


In [24]:
# This describes the non-numerical columns:  

storedf.describe(include=[object])

# At a glance, there are 49 states, 4 regions and 531 cities included.
# The top sales are in New York City and the State of California.
# There are 4 unique ship modes and the  top ship mode is the standard class and the top type of customer is a consumer.
# The top category is office supplies and the top sales category is of a low value.

,Ship Mode,Segment,City,State,Region,Category,Sub-Category,Product Name,Sales Category
count,9994,9994,9994,9994,9994,9994,9994,9994,9994
unique,4,3,531,49,4,3,17,1850,3
top,Standard Class,Consumer,New York City,California,West,Office Supplies,Binders,Staple envelope,Low value
freq,5968,5191,915,2001,3203,6026,1523,48,8233


In [25]:
# This checks if there are any blank fields. All columns seem to have data, there are no missing fields.

print(storedf.isnull().sum())


Order Date               0
Ship Date                0
Order Processing Days    0
Month                    0
Year                     0
Ship Mode                0
Segment                  0
City                     0
State                    0
Region                   0
Category                 0
Sub-Category             0
Product Name             0
Sales                    0
Quantity                 0
SP per unit              0
Discount                 0
Profit                   0
Profit Margin (%)        0
Sales Category           0
dtype: int64


In [26]:
# These can be used to get the value counts of different columns:  
storedf["Category"].value_counts()
# This shows that the most lines of sales are office supplies.

Category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64

In [27]:
# This can be used to calculate the number in each sub-category:  
 
storedf["Sub-Category"].value_counts()

# This shows that the most sales are on binders and the least sales are copiers

Sub-Category
Binders        1523
Paper          1370
Furnishings     957
Phones          889
Storage         846
Art             796
Accessories     775
Chairs          617
Appliances      466
Labels          364
Tables          319
Envelopes       254
Bookcases       228
Fasteners       217
Supplies        190
Machines        115
Copiers          68
Name: count, dtype: int64

In [ ]:
# Next I will look at the top 5 products by name of product.
groupproduct = storedf.groupby('Product Name').agg(total_sales=('Sales', 'sum'), total_profit=('Profit', 'sum')).reset_index()
top_5_products = groupproduct.sort_values(by='total_sales', ascending=False).head(5)
print(top_5_products[['Product Name', 'total_sales', 'total_profit']])

# This shows us that the Canon copier has the highest sales and a decent profit.
# On the other hand the Cisco Telepresence System have high sales but a negative margin.
# The Hon 5400 Series Task Chairs appeaar to be sold at cost as there is no profit.

                                          Product Name  total_sales  \
404              Canon imageCLASS 2200 Advanced Copier    61,599.82   
650  Fellowes PB500 Electric Punch Plastic Comb Bin...    27,453.38   
444  Cisco TelePresence System EX90 Videoconferenci...    22,638.48   
786       HON 5400 Series Task Chairs for Big and Tall    21,870.58   
686         GBC DocuBind TL300 Electric Binding System    19,823.48   

     total_profit  
404     25,199.93  
650      7,753.04  
444     -1,811.08  
786          0.00  
686      2,233.51  


In [31]:
# Next I will look at the top Sales Categories
groupproduct = storedf.groupby('Category').agg(total_sales=('Sales', 'sum'), total_profit=('Profit', 'sum')).reset_index()
top_categories = groupproduct.sort_values(by='total_sales', ascending=False).head(3)
print(top_categories[['Category', 'total_sales', 'total_profit']])
# Technology has the highest sales and the highest profit.  
# Furniture is the second highest sales category but has a low profit margin.   



          Category  total_sales  total_profit
2       Technology   836,154.03    145,454.95
0        Furniture   741,999.80     18,451.27
1  Office Supplies   719,047.03    122,490.80


In [38]:
# Next I will look at the top Sales Sub-Categories
groupproduct = storedf.groupby('Sub-Category').agg(total_sales=('Sales', 'sum'), total_profit=('Profit', 'sum')).reset_index()
top_sub_categories = groupproduct.sort_values(by='total_sales', ascending=False).head(10)
print(top_sub_categories[['Sub-Category', 'total_sales', 'total_profit']])

# Phones are the highest sales sub-category and have a good profit margin 
# Copiers have a high sales value and a high profit margin.  
# Note that tables and bookcases have a reasonable sales value but a negative margin which would tie in with the furniture category above, this could indicate that some of these negative margin items are bringing down the overall profit.  

   Sub-Category  total_sales  total_profit
13       Phones   330,007.05     44,515.73
5        Chairs   328,449.10     26,590.17
14      Storage   223,843.61     21,278.83
16       Tables   206,965.53    -17,725.48
3       Binders   203,412.73     30,221.76
11     Machines   189,238.63      3,384.76
0   Accessories   167,380.32     41,936.64
6       Copiers   149,528.03     55,617.82
4     Bookcases   114,880.00     -3,472.56
1    Appliances   107,532.16     18,138.01


## Research

1. [Datetime](https://www.statology.org/convert-columns-to-datetime-pandas/) - Check to see how to use datetime function to convert dates so they can be used for analysis.  
2. [Chardet](https://stackoverflow.com/questions/54389780/using-chardet-to-detect-encoding)The file looked okay but there may have been unreadable characters, meaning the file was encoded with one format and your application is trying to read it using a different encoding format.  
3. [Adding Columns](https://realpython.com/pandas-dataframe/#inserting-and-deleting-columns) - adding a column to my file. I used this tutorial to add some columns relevant to my analysis.  
4. [Datetime](https://stackoverflow.com/questions/69375868/extract-month-from-datetime-column-in-pandas-dataframe) - extracting a month from the order date.  
5. [Adding Columns](https://stackoverflow.com/questions/59642338/creating-new-column-based-on-condition-on-other-column-in-pandas-dataframe) - Adding a column based on data in another column, in this case, using the Selling price per unit column, and categorising the sales by value.  
6. [Describe](https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/percentiles/a/percentiles-intro) - I used this to help understand the different columns in the describe command.
7. [Pre-processing Data](https://www.kdnuggets.com/7-steps-to-mastering-data-cleaning-with-python-and-pandas) - reviewing and cleaning data.
8. [Regex](https://stackoverflow.com/questions/40121822/extracting-year-from-string-in-python) - I checked if I could use an example to extract the year and enter it in a different column but as I already converted the columns to datetime, it made more sense to extract it using dt.year.  
9. [Groupby](https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/) - using groupby function to calculate top sales products and other categories.